In [73]:
import os

import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [79]:
data = pd.read_csv("/content/IMDB Dataset.csv")


In [80]:
data.shape

(9509, 2)

In [81]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [82]:
data.tail()

,review,sentiment
9504,Michael Winner is probably best known for his ...,positive
9505,Outstanding film of 1943 with Paul Lukas givin...,positive
9506,I remember being forced (yes--literally FORCED...,negative
9507,"After some quite OK Dutch action flicks, like ...",negative
9508,Thelma Ritter did steal the picture. I just fi...,positive


In [83]:
data["sentiment"].value_counts()

sentiment
positive    4825
negative    4684
Name: count, dtype: int64

In [84]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [85]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [86]:
data["sentiment"].value_counts()

sentiment
1    4825
0    4684
Name: count, dtype: int64

In [88]:
# split data into training data and test data
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)

print(train_data.shape)
print(test_data.shape)

(8558, 2)
(951, 2)


Data Pre-processing:

In [89]:
# Tokenize text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [90]:
print(X_train)

[[  14   16   44 ...  103    1   19]
 [   0    0    0 ...   34  958    2]
 [   0    0    0 ...   14    1  223]
 ...
 [   0    0    0 ...  313  331  238]
 [   0    0    0 ...    4    1 3821]
 [   0    0    0 ...  238   41    4]]


In [91]:
print(X_test)

[[   0    0    0 ...   61   16  323]
 [   0    0    0 ...  805  519 1236]
 [   0    0    0 ...   44    5   81]
 ...
 [   0    0    0 ...   90  455   12]
 [   0    0    0 ...    8  980 2612]
 [   0    0    0 ... 1106 1577  432]]


In [92]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [93]:
print(Y_train)

8864    0
554     0
3333    0
511     0
1315    1
       ..
5734    1
5191    0
5390    0
860     0
7270    0
Name: sentiment, Length: 8558, dtype: int64


In [94]:
print(Y_test)

2441    1
7149    1
3133    0
9407    1
683     1
       ..
8105    1
2935    1
1020    0
6517    0
3458    0
Name: sentiment, Length: 951, dtype: int64


LSTM - Long Short-Term Memory

In [95]:
# build the model

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

In [96]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 200, 128)          640000    
                                                                 
 lstm_2 (LSTM)               (None, 128)               131584    
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 771713 (2.94 MB)
Trainable params: 771713 (2.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [97]:
# compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

Training the model:

In [99]:
model.fit(X_train, Y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
107/107 [==============================] - 96s 882ms/step - loss: 0.0567 - accuracy: 0.9810 - val_loss: 0.9973 - val_accuracy: 0.7699
Epoch 2/10
107/107 [==============================] - 92s 856ms/step - loss: 0.0435 - accuracy: 0.9855 - val_loss: 0.9061 - val_accuracy: 0.8236
Epoch 3/10
107/107 [==============================] - 89s 831ms/step - loss: 0.0113 - accuracy: 0.9974 - val_loss: 1.0194 - val_accuracy: 0.8259
Epoch 4/10
107/107 [==============================] - 91s 849ms/step - loss: 0.0218 - accuracy: 0.9950 - val_loss: 0.9812 - val_accuracy: 0.7815
Epoch 5/10
107/107 [==============================] - 91s 849ms/step - loss: 0.0167 - accuracy: 0.9947 - val_loss: 1.0485 - val_accuracy: 0.8195
Epoch 6/10
107/107 [==============================] - 91s 849ms/step - loss: 0.0073 - accuracy: 0.9984 - val_loss: 1.0980 - val_accuracy: 0.8189
Epoch 7/10
107/107 [==============================] - 91s 852ms/step - loss: 0.0262 - accuracy: 0.9907 - val_loss: 1.0796 - val_ac

Model Evaluation:

In [100]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

30/30 [==============================] - 2s 75ms/step - loss: 1.1397 - accuracy: 0.8233
Test Loss: 1.1396797895431519
Test Accuracy: 0.8233438730239868


Build a predictive system:

In [107]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [106]:
# example usage
new_review = "This movie was very good."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 54ms/step
The sentiment of the review is: positive


In [108]:
# example usage
new_review = "This movie was not that good"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 80ms/step
The sentiment of the review is: negative


In [109]:
# example usage
new_review = "This movie was ok but not that good."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 52ms/step
The sentiment of the review is: negative
